In [69]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import svm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
import pickle


def plot_confusion_matrix(y_true, y_pred, label,
                          cmap=plt.cm.Blues):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data

    print(cm)

    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=label, yticklabels=label,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


import pandas as pd

In [70]:
c = [1, 10, 100, 1000]
gamma = [1e-3, 1e-4,.04]
features = ['Date', 'AORD_monthly_adj_closed', '10Y_Bond_Yields', '3M_Bond_Yields',
                         '3M_10Y_Treasury_Spread', 'Unemployment_Rate', 'Recession',
                         'Recession_within_6M',
                         'Recession_within_12M', 'Recession_within_24M']
df = pd.read_csv('C:/Users/Rohit/Desktop/Deakin Assignments/Trimester 2 - March/Team Project Management - A/Project/companyandngo/Data/Datasets/AU Final.csv')[::-1]
predicted_probability=pd.DataFrame()
past_predict = pd.DataFrame()
test_prob = pd.DataFrame()
recession_data = df[features]
#df_features.join(pd.get_dummies(df['Recession']))
output_feature =['Recession_within_6M','Recession_within_12M']

In [71]:
x_training_data=recession_data[recession_data['Date'] <= '2018-02-01']
x_testing_data = recession_data[recession_data['Date'] >= '2018-02-01']

In [72]:
c

[1, 10, 100, 1000]

In [73]:
X_train, X_test, y_train, y_test = train_test_split(x_training_data, x_training_data.iloc[:,7:], test_size=0.30)

In [74]:
x_testing_data['Recession_within_24M']

426    1
425    0
419    0
418    0
417    0
      ..
9      0
8      0
7      0
6      0
5      0
Name: Recession_within_24M, Length: 247, dtype: int64

In [75]:
for i in output_feature:
    svm_ac = {}
    for x, y in [(x, y) for x in c for y in gamma]:
        k = x, y
        # initalize the'model' and passing the SVM Classifier with respect to C and gamma value
        model = svm.SVC(C=x, kernel='rbf', gamma=y, probability=True,
                        tol=1e-3,
                        class_weight='balanced')
        accuracy = cross_val_score(model, X_train.iloc[:,1:6], y_train[i], cv=10, scoring='accuracy')
        svm_ac[k] = np.mean(accuracy)
        print("\n{:20} {:25}".format('Parameter', 'Average accuracy'))
        print('{}\t{:25}'.format(k, np.mean(accuracy)))
    print(f"\n----{i}------")
    acc_score = max(svm_ac, key=svm_ac.get)
    print(f"The best tuning parameter{acc_score}")
    svc_model = svm.SVC(C=acc_score[0], kernel='rbf', gamma=acc_score[1], probability=True,
                        tol=1e-3, random_state=123,
                        class_weight='balanced')
   
    # fitting the model with training data
    svc_model.fit(X_train.iloc[:,1:6], y_train[i])
    # predicting the model with the test data
    y_predict = svc_model.predict(X_test.iloc[:,1:6])

    # evaluating the accuracy with the ground truth and predicted value
    model_acc = accuracy_score(y_test[i], y_predict)

    print("Model Accuracy is: {}".format(model_acc))
    # classification_report((y_true, y_pred)
    predicted_probability[i+"_probability"] = svc_model.predict_proba(x_testing_data.iloc[:,1:6])[:,1]
    past_predict[i+"_probability"]=svc_model.predict_proba(X_train.iloc[:,1:6])[:,1]
    test_prob[i+"_probability"]=svc_model.predict_proba(X_test.iloc[:,1:6])[:,1]
    filename = i+'.sv'
    pickle.dump(svc_model, open(filename, 'wb'))
      


Parameter            Average accuracy         
(1, 0.001)	       0.5480769230769231

Parameter            Average accuracy         
(1, 0.0001)	       0.5820512820512821

Parameter            Average accuracy         
(1, 0.04)	       0.4833333333333334

Parameter            Average accuracy         
(10, 0.001)	      0.49102564102564095

Parameter            Average accuracy         
(10, 0.0001)	       0.5564102564102564

Parameter            Average accuracy         
(10, 0.04)	      0.46794871794871795

Parameter            Average accuracy         
(100, 0.001)	        0.532051282051282

Parameter            Average accuracy         
(100, 0.0001)	       0.5878205128205128

Parameter            Average accuracy         
(100, 0.04)	      0.46794871794871795

Parameter            Average accuracy         
(1000, 0.001)	       0.5166666666666666

Parameter            Average accuracy         
(1000, 0.0001)	        0.516025641025641

Parameter            Average accuracy         
(

In [76]:
predicted_probability['Date']= x_testing_data['Date'].tolist()
predicted_probability

,Recession_within_6M_probability,Recession_within_12M_probability,Date
0,0.449888,0.537253,8/1/1984
1,0.449921,0.537374,9/1/1984
2,0.500000,0.522494,3/1/1985
3,0.493111,0.483236,4/1/1985
4,0.493451,0.484362,5/1/1985
...,...,...,...
242,0.460419,0.384816,5/1/2019
243,0.461603,0.477717,6/1/2019
244,0.457559,0.419814,7/1/2019
245,0.461374,0.479163,8/1/2019


In [77]:
past_predict['Date']= X_train['Date'].tolist()
test_prob['Date']=X_test['Date'].tolist()

In [78]:
past_predict.append(test_prob)

,Recession_within_6M_probability,Recession_within_12M_probability,Date
0,0.465560,0.360084,11/1/2018
1,0.440718,0.374666,12/1/2006
2,0.459740,0.374620,12/1/2004
3,0.462308,0.543287,2/1/1989
4,0.477414,0.543894,2/1/1990
...,...,...,...
49,0.461090,0.391776,10/1/2001
50,0.500000,0.569729,2/1/1988
51,0.449049,0.379660,10/1/2008
52,0.463649,0.568096,1/1/1987


In [79]:
past_predict.append(predicted_probability)

,Recession_within_6M_probability,Recession_within_12M_probability,Date
0,0.465560,0.360084,11/1/2018
1,0.440718,0.374666,12/1/2006
2,0.459740,0.374620,12/1/2004
3,0.462308,0.543287,2/1/1989
4,0.477414,0.543894,2/1/1990
...,...,...,...
242,0.460419,0.384816,5/1/2019
243,0.461603,0.477717,6/1/2019
244,0.457559,0.419814,7/1/2019
245,0.461374,0.479163,8/1/2019


In [80]:
file=['Recession_within_6M.sv','Recession_within_12M.sv']

for i in file:
    svmc = pickle.load(open(i, 'rb'))
    print(svmc.predict_proba(test.iloc[:,1:6])[:,1])


[0.44988842]
[0.53725345]


In [81]:
test=recession_data[recession_data['Date'] == '8/1/1984']

test

,Date,AORD_monthly_adj_closed,10Y_Bond_Yields,3M_Bond_Yields,3M_10Y_Treasury_Spread,Unemployment_Rate,Recession,Recession_within_6M,Recession_within_12M,Recession_within_24M
426,8/1/1984,733.200012,12.8,11.56,1.24,8.82942,0,0,1,1


In [82]:
X_train,X_test, y_train, y_test = train_test_split(x_training_data, x_training_data.iloc[:,7:], test_size=0.30)

In [83]:

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for i in output_feature:
    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='accuracy'
    )
    clf.fit(X_train.iloc[:,1:6], y_train[i])

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    #y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print(f"------{i}---------")
    print()

Best parameters set found on development set:

{'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}

Grid scores on development set:

0.628 (+/-0.156) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.675 (+/-0.090) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.620 (+/-0.130) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.651 (+/-0.113) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.619 (+/-0.114) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.651 (+/-0.134) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.619 (+/-0.114) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.667 (+/-0.157) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.500 (+/-0.168) for {'C': 1, 'kernel': 'linear'}
0.493 (+/-0.136) for {'C': 10, 'kernel': 'linear'}
0.453 (+/-0.122) for {'C': 100, 'kernel': 'linear'}
0.469 (+/-0.150) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.



NameError: name 'y_true' is not defined